In [29]:

#csvfile = r'C:\Users\roumba\Documents\Software\deep-events\original_data\220915_mtstaygold_cos7_ZEISS_fl\labels1f.csv'
#imagefile= r'C:\Users\roumba\Documents\Software\deep-events\original_data\220915_mtstaygold_cos7_ZEISS_fl\220915_cos7_mitostaygold_GFP_1.ome.tif'

from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.io
import plotly.express as px
from scipy import ndimage as ndi
import tensorflow
import imageio
from scipy import signal
from pathlib import Path
import tensorflow_probability as tfp
import tensorflow as tf
from tqdm import tqdm  
from os import path
import myfunctions
from myfunctions import event_separation , image_crop_save_gauss , image_crop_save , image_crop_negative, poi
import tifffile


## Input data from a folder ##

In [30]:
files_dir = r'C:\Users\roumba\Documents\Software\deep-events\original_data'
images_dir = '220915_mtstaygold_cos7_ZEISS_fl'
joined_path = os.path.join(files_dir, images_dir)
sigma=(8,8)
size=(2048,2048)
img,input_name,output_name,datacsv={},{},{},{}

for input_file in os.listdir(joined_path):
        joined_file_path = os.path.join(files_dir, images_dir, input_file)
        if '.tif' in input_file:
            date, cell_type, dye, bf_fl, index  = input_file.split('_')
            number, ome, tiff= index.split('.')
            img[int(number)-1] = Image.open(joined_file_path)
            input_name[int(number)-1]= f'points_{date}_{cell_type}_{bf_fl}_{number}.tiff'
            output_name[int(number)-1]= f'image_{date}_{cell_type}_{bf_fl}_{number}'
            print('Loaded image:'+input_file)
        else:
            labels, index, bf_fl  = input_file.split('_')
            datacsv[int(index)-1] = pd.read_csv(joined_file_path)
            print('Loaded csv:'+ input_file)
max_number=int(number)        


Loaded image:220915_cos7_mitostaygold_GFP_1.ome.tif
Loaded image:220915_cos7_mitostaygold_GFP_2.ome.tif
Loaded image:220915_cos7_mitostaygold_GFP_3.ome.tif
Loaded csv:labels_1_f.csv
Loaded csv:labels_2_f.csv
Loaded csv:labels_3_f.csv


## Get all Gaussian Points of Interest ##

In [32]:
for x in range(max_number):
    csv=datacsv[x]
    in_name=input_name[x]
    poi(csv,in_name,sigma,size)

100%|██████████| 32/32 [00:07<00:00,  4.39it/s]


## Cut Images and Gaussian POIs ##

In [33]:
base_dir= r'C:\Users\roumba\Documents\Software\deep-events'
for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    out_name=output_name[x]
    in_name=input_name[x]

    list1=event_separation(csv)
    l=len(list1)
    image_crop_save(l,list1, csv, im, out_name)

    gauss=base_dir+f'\{in_name}'
    gauss_image=Image.open(gauss)
    image_crop_save_gauss(l,list1, csv, gauss_image, out_name)
    


## Cut Negative Frames ##

In [ ]:
for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    out_name=output_name[x]
    in_name=input_name[x] 
    image_crop_negative(l,list1, csv, im, out_name)

## Save data in correct folders ##

In [34]:
#training_dir=r'C:\Users\roumba\Documents\Software\deep-events\training_data'
#for input_file in os.listdir(base_dir):
        # if 'neg' and 'image' in input_file:
        #     title,date,cell,microscope,videonumber,index=input_file.split('_')
        #     old_dir=base_dir+ f'\{input_file}'
        #     new_dir=
        #     os.replace('old_directory/test_file.txt', 'new_directory/test_file.txt')
        # else:
        #     title,date,cell,microscope,videonumber,index=input_file.split('_')
        #     labels, index, bf_fl  = input_file.split('_')
        #     datacsv[int(index)-1] = pd.read_csv(joined_file_path)
        #     print('Loaded csv:'+ input_file)